In [1]:
import yfinance as yf
import requests, zipfile
from datetime import date, timedelta
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType, DateType, TimestampType, FloatType
from pyspark.sql.functions import udf, col, lit, when, trim
from pyspark import SparkFiles

#### call lib

In [2]:
%run ./work/lib.ipynb

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6476b6ac-11eb-4223-81bf-acef600dcd06;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 245ms :: artifacts dl 7ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runt

#### request cvm

In [3]:
url = "https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv"
spark.sparkContext.addFile(url)

cadastro_cvm_df = \
    spark \
        .read \
        .options(header='True', delimiter=';', encoding='iso-8859-1', inferSchema= True) \
        .csv(SparkFiles.get("cad_cia_aberta.csv"))

cadastro_cvm_df = \
    cadastro_cvm_df \
        .select(
            trim('CNPJ_CIA').alias('cnpj'),
            col('DENOM_SOCIAL').alias('company_name'),
            col('DT_REG').cast('date').alias('register_date'),
            col('DT_CANCEL').cast('date').alias('inactive_date'),
            when(col('SIT') == 'ATIVO', lit('Active')).otherwise(lit('Inactive')).alias('status'),
            trim('SETOR_ATIV').alias('segment'),
            col('PAIS').alias('country'),
            col('UF').alias('state'),
            col('MUN').alias('city')
        )

##### stock df

In [4]:
stocks_df = \
    spark \
        .read \
        .format('csv') \
        .options(header='True', delimiter=';', encoding='iso-8859-1') \
        .csv('{}/extract/stock_list.csv'.format(LAKE_HOME)) \
        .select(
            trim('CNPJ').alias('cnpj'),
            col('TICKER').alias('ticker')
        )

In [5]:
cadastro_cvm_df.where(col('cnpj') == '33.000.167/0001-01').show()

+------------------+--------------------+-------------+-------------+------+--------------+-------+-----+--------------+
|              cnpj|        company_name|register_date|inactive_date|status|       segment|country|state|          city|
+------------------+--------------------+-------------+-------------+------+--------------+-------+-----+--------------+
|33.000.167/0001-01|PETRÓLEO BRASILEI...|   1977-07-20|         null|Active|Petróleo e Gás| BRASIL|   RJ|RIO DE JANEIRO|
|33.000.167/0001-01|PETRÓLEO BRASILEI...|   1977-07-20|         null|Active|Petróleo e Gás| BRASIL|   RJ|RIO DE JANEIRO|
+------------------+--------------------+-------------+-------------+------+--------------+-------+-----+--------------+



##### join dfs

In [14]:
dim_ticker = \
    stocks_df \
        .join(
            cadastro_cvm_df,
            on=['cnpj'],
            how='inner'
        ) \
        .dropDuplicates(['ticker'])

##### Write to DW

In [16]:
df = dim_ticker

write_to_dw(df, 'dim_ticker')